# Deep Neural Network for MNIST Classification

The dataset is called MNIST and refers to handwritten digit recongnition.

The dataset provides 70000 images (28x28 pixels) of handwritten digits (1 digit per image).

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits, this is a classification problem with 10 classes. 

Goals is to build a neural network with 2 hidden layers. 

## Import the relevant packages

In [12]:
import numpy as np
import tensorflow as tf
#pip install tensorflow_datasets
import tensorflow_datasets as tfds

## Disable Python warnings

In [13]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

## Data

In [14]:
#tfds.load loads a datasets
#with_info = True will provide us with a tuple containing information about the version, features, number of samples
#as_supervised = True will laod the dataset in a 2-tuple structure (input, target)
mnist_dataset, mnist_info = tfds.load(name = "mnist", with_info = True, as_supervised = True)

#extract the training and testing dataset with build references
mnist_train, mnist_test = mnist_dataset["train"], mnist_dataset["test"]

#TensorFlow has training and testing datasets but no validation sets, thus we need to split it on our own

#defining the number of validation samples as a % of the train samples
num_validation_samples = 0.1 * mnist_info.splits["train"].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64) #convert to integer

#store the number of test samples in variable
num_test_samples = mnist_info.splits["test"].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64) #convert to integer

#define a function called scale; take an MNIST image and label
def scale(image, label):
    image = tf.cast(image, tf.float32) #value make it as float
    image /= 255. #divide each element by 255 as the possible values for the inputs are 0 to 255
    return image, label

#.map() allow us to apply custom transformation to a given dataset
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

#shuffle the data 

#set buffer size parameter for cases when dealing with enormous datasets
#can't shuffle the whole dataset in one go becasue can't fit it all in memory
BUFFER_SIZE = 10000

#.shuffle() method to shuffle
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

#.take() method to take many samples
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

#.skip() skip as many samples as there are in the validation dataset
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

#set batch size
BATCH_SIZE = 100

#batch the data
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_test_samples)
test_data = test_data.batch(num_test_samples)

#take next batch; also the only batch as we set as_supervized = True, only got 2-tuple structure 
validation_inputs, validation_targets = next(iter(validation_data))

2023-11-15 13:45:39.632011: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


## Model

### Outline the model 

In [49]:
input_size = 784
output_size = 10
hidden_layer_size = 50

#define how the model will look like
model = tf.keras.Sequential([
                              #each observation is 28x28x1 pixels, therefore is tensor of rank 3
                              #don't know CNNs yet, need to flatten the images; 
                              #use the Flatten method to converts multi-dimensional arrays into flattened one-dimensional arrays
                              tf.keras.layers.Flatten(input_shape = (28, 28, 1)), #input layer
                              #use the Dense method
                              tf.keras.layers.Dense(hidden_layer_size, activation = "relu"), #1st hidden layer
                              tf.keras.layers.Dense(hidden_layer_size, activation = "relu"), #2nd hidden layer
                              #activate final layer with softmax
                              tf.keras.layers.Dense(output_size, activation = "softmax") #output layer
                            ])

### Choose the optimizer and the loss function

In [50]:
#define the optimizer, loss and metrics
model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

### Training

In [52]:
#determine the maximum number of epochs
NUM_EPOCHS = 30

#fit the model by training data, total number of epochs and the validation data
model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), verbose = 2)

Epoch 1/30
540/540 - 1s - loss: 0.0172 - accuracy: 0.9947 - val_loss: 0.0295 - val_accuracy: 0.9902 - 672ms/epoch - 1ms/step
Epoch 2/30
540/540 - 1s - loss: 0.0188 - accuracy: 0.9940 - val_loss: 0.0350 - val_accuracy: 0.9887 - 606ms/epoch - 1ms/step
Epoch 3/30
540/540 - 1s - loss: 0.0153 - accuracy: 0.9951 - val_loss: 0.0206 - val_accuracy: 0.9938 - 621ms/epoch - 1ms/step
Epoch 4/30
540/540 - 1s - loss: 0.0150 - accuracy: 0.9948 - val_loss: 0.0132 - val_accuracy: 0.9952 - 626ms/epoch - 1ms/step
Epoch 5/30
540/540 - 1s - loss: 0.0152 - accuracy: 0.9948 - val_loss: 0.0177 - val_accuracy: 0.9940 - 594ms/epoch - 1ms/step
Epoch 6/30
540/540 - 1s - loss: 0.0111 - accuracy: 0.9966 - val_loss: 0.0130 - val_accuracy: 0.9967 - 606ms/epoch - 1ms/step
Epoch 7/30
540/540 - 1s - loss: 0.0131 - accuracy: 0.9956 - val_loss: 0.0133 - val_accuracy: 0.9955 - 630ms/epoch - 1ms/step
Epoch 8/30
540/540 - 1s - loss: 0.0102 - accuracy: 0.9968 - val_loss: 0.0120 - val_accuracy: 0.9957 - 634ms/epoch - 1ms/step


### Test the model 

In [55]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 132ms/step - loss: 0.1667 - accuracy: 0.9731


In [56]:
#apply formatting
print("Test loss: {0:.2f}. Test accuracy: {1:.2f}%".format(test_loss, test_accuracy * 100.))

Test loss: 0.17. Test accuracy: 97.31%


The final test accuracy should be roughly around 97%.